In [54]:
# # 페이지가 로딩될 때까지 기다림
# time.sleep(10)

# # class가 mat-table인 table을 찾아서 출력
# table = driver.find_element('class name', 'mat-table')

# # 테이블 HTML 가져오기
# table_html = table.get_attribute('outerHTML')

# # BeautifulSoup을 사용하여 테이블 파싱
# soup = BeautifulSoup(table_html, 'html.parser')
# table_soup = soup.find('table')

# # 테이블을 DataFrame으로 변환
# df = pd.read_html(str(table_soup))[0]

# # Time Temperature Condition 은 남기고 Dew Point Humidity  Wind Wind Speed Wind Gust Pressure  Precip. 은 drop
# df = df.drop(columns=['Dew Point', 'Humidity', 'Wind', 'Wind Speed', 'Wind Gust', 'Pressure', 'Precip.'])

# # "date" column 추가 
# df['date'] = '2024-1-1'

# # DataFrame 출력
# print(df)

In [55]:
from selenium import webdriver
import time
import pandas as pd
from bs4 import BeautifulSoup

# https://www.wunderground.com/history/daily/us/ny/new-york-city/KLGA/date/2024-1-1 을 여는 selenium 코드

def get_weather_data(month, day):
    driver = webdriver.Chrome()
    driver.get(f"https://www.wunderground.com/history/daily/us/ny/new-york-city/KLGA/date/2024-{month}-{day}")
    time.sleep(10)
    table = driver.find_element('class name', 'mat-table')
    table_html = table.get_attribute('outerHTML')
    soup = BeautifulSoup(table_html, 'html.parser')
    table_soup = soup.find('table')
    df = pd.read_html(str(table_soup))[0]
    df = df.drop(columns=['Dew Point', 'Humidity', 'Wind', 'Wind Speed', 'Wind Gust', 'Pressure', 'Precip.'])
    df['date'] = f'2024-{month}-{day}'
    
    # 'Time'컬럼이 nan인 row를 제거
    df = df.dropna(subset=['Time'])

    # 마지막 행의 'Time' 컬럼 값에 'AM'이 포함되어있으면 그 행의 'date' 컬럼 값을 다음날로 변경 
    if 'AM' in df.iloc[-1]['Time']:
        df.iloc[-1, df.columns.get_loc('date')] = f'2024-{month}-{day+1}'
    
    if 'AM' in df.iloc[-2]['Time']:
        df.iloc[-2, df.columns.get_loc('date')] = f'2024-{month}-{day+1}'
    
    
    
    driver.quit()
    return df

df = pd.DataFrame(columns=['Time', 'Temperature', 'Condition', 'date'])

# df에 df를 이어붙임
for month in range(1, 2):
    for day in range(1, 32):
        try:
            df = pd.concat([df, get_weather_data(month, day)], ignore_index=True)
        except:
            pass

print(df)




/var/folders/v4/6tjs7h4921v9vrm_844m1clw0000gn/T/ipykernel_62151/3145647726.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table_soup))[0]


        Time Temperature      Condition      date
0    1:51 AM       43 °F         Cloudy  2024-1-1
1    2:51 AM       43 °F         Cloudy  2024-1-1
2    3:51 AM       42 °F         Cloudy  2024-1-1
3    4:51 AM       42 °F         Cloudy  2024-1-1
4    5:51 AM       42 °F         Cloudy  2024-1-1
5    6:51 AM       42 °F         Cloudy  2024-1-1
6    7:13 AM       42 °F     Light Rain  2024-1-1
7    7:51 AM       41 °F     Light Rain  2024-1-1
8    8:51 AM       40 °F     Light Rain  2024-1-1
9    9:51 AM       40 °F     Light Rain  2024-1-1
10  10:34 AM       41 °F  Mostly Cloudy  2024-1-1
11  10:51 AM       41 °F  Mostly Cloudy  2024-1-1
12  11:51 AM       44 °F  Mostly Cloudy  2024-1-1
13  12:51 PM       45 °F  Mostly Cloudy  2024-1-1
14   1:51 PM       45 °F  Mostly Cloudy  2024-1-1
15   2:51 PM       45 °F  Mostly Cloudy  2024-1-1
16   3:51 PM       45 °F  Mostly Cloudy  2024-1-1
17   4:51 PM       45 °F  Mostly Cloudy  2024-1-1
18   5:51 PM       45 °F  Mostly Cloudy  2024-1-1


/var/folders/v4/6tjs7h4921v9vrm_844m1clw0000gn/T/ipykernel_62151/3145647726.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table_soup))[0]


In [56]:
# df의 colunm별 type 확인
print(df['Condition'][0])
print(df['Time'][0])

Cloudy
1:51 AM


In [57]:
def am_pm_to_24(time):
    time = str(time)
    hour, minute, AM_PM = time.replace(':', ' ').split()
    hour = int(hour)%12
    
    if AM_PM == 'PM':
        hour += 12
        return f"{hour:02d}:{minute}:00"
    else:
        return f"{hour:02d}:{minute}:00"

# df의 Time column 의 시간 1:51 PM 에서 24시간 형식으로 바꿈
df['Time'] = df['Time'].apply(am_pm_to_24)

# df의 Time column에서 시간만 남긴 컬럼을 추가
df['hour'] = df['Time'].apply(lambda x: int(x.split(':')[0]))

# df 의 Temperature column의 값 "43 °F" 형태에서 숫자만 남김
df['Temperature'] = df['Temperature'].apply(lambda x: int(x.split()[0]))

# df의 Temperature column의 title을 "Temperature(F)"로 바꿈
df = df.rename(columns={'Temperature': 'Temperature (F)'})

# hour과 date로 groupby하여 Temperature의 평균을 구함 Time, Condition, date, hour 은 first로 함
df = df.groupby(['date', 'hour']).agg({'Time': 'first', 
                                       'Temperature (F)': 'mean', 
                                       'Condition': 'first'}).reset_index()

# df 정렬 date 오름차순, hour 오름차순
df = df.sort_values(by=['date', 'hour'])

# df를 csv로 저장
df.to_csv('weather.csv', index=False)


In [58]:


df


,date,hour,Time,Temperature (F),Condition
0,2024-1-1,1,01:51:00,43.0,Cloudy
1,2024-1-1,2,02:51:00,43.0,Cloudy
2,2024-1-1,3,03:51:00,42.0,Cloudy
3,2024-1-1,4,04:51:00,42.0,Cloudy
4,2024-1-1,5,05:51:00,42.0,Cloudy
5,2024-1-1,6,06:51:00,42.0,Cloudy
6,2024-1-1,7,07:13:00,41.5,Light Rain
7,2024-1-1,8,08:51:00,40.0,Light Rain
8,2024-1-1,9,09:51:00,40.0,Light Rain
9,2024-1-1,10,10:34:00,41.0,Mostly Cloudy
